[Go to this directory index](./)

Contents
  - [More optimization trials¶](#More-optimization-trials)
      - [Trial 2.1¶](#Trial-2.1)
      - [Trial 2.2¶](#Trial-2.2)
        - [Aside¶](#Aside)
      - [Trial 2.3¶](#Trial-2.3)

<IPython.core.display.Javascript object>

# More optimization trials

Previously, I tried throwing a bunch of optimizers at my model. I liked the results from the scipy.optimize routines COBYLA and BFGS. Let's try adjusting how we handle the constraints, and see if we get better results.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
import numpy
from IPython.display import HTML, SVG

In [3]:
matplotlib.use('svg')
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.sans-serif'] = 'Arial'

In [10]:
from io import BytesIO
def pltsvg():
    imgdata = BytesIO()
    plt.savefig(imgdata)
    imgdata.seek(0)
    display(SVG(data=imgdata.read()))

### Trial 2.1

In general, here is how we compute the modified objective function. GenOpt manual suggests adding functions to the objective, namely separate barrier (B) and penalty (P) functions. The objective function is, with step number as $k$,

$$ Objective = -Q_{cooling}(x) + \mu_1 B(x) + \mu_2 P(x) $$

$$ \mu_1 = k^{-2}$$
$$ B(x) = \left(\sum_{j=1}^{N_B} g_j(x) \right) ^ {-1} $$

$$\mu_2 = k^{2}$$
$$ P(x) = \sum_{j=N_B+1}^{N_B + N_P} g_j(x)$$

GenOpt implements a mechanism to pass in the step number. However, using the scipy.optimize routines, passing in the step number is a challenge, so the `Problem` object should probably use its own counter of function calls. This could be quirky, since then the objective function is changing with each call. Some routines allow for termination after a number of steps, so we could run a while, increment step number, then continue.

But we need to make some decisions.

- Which constraints should be handled by barriers, and which by penalties?
  - Use barriers for hard constraints, eg. required for feasibility of the chiller model
  - Use penalties for soft constraints, eg. heat exchange feasibility that can be violated now and satisfied later
- With scipy.optimize, we can directly apply constraints on the input variables (such as $T_1 > T_2$)



In [ ]:
import ammonia1
import system_aqua1
import scipy.optimize

class Problem_2_1:
    def __init__(self, bdry, UAgoal, mu=0.1):
        self.bdry = bdry
        self.UAgoal = UAgoal
        self.mu = 0.1
        self.Ncons = 5
        # Soft constraints mode: this is sent to minimizer
        self.constraints = [{'type': 'ineq',
                             'fun': self.constraint,
                             'args': (i,)
                            } for i in range(self.Ncons)]
            
    def objective(self, xC, stepNumber=1):
        try:
            ch = system_aqua1.makeChiller(xC)
            sys = system_aqua1.System(self.bdry, ch)
            Q = sys.chiller.Q_evap
            B = 0
            P = 0
            return -Q + B + P
        except:
            return numpy.inf
    
    def constraint(self, x, *args):
        i, = args
        cons = [x[0],
                x[2] - x[1],
                x[3] - x[2],
                x[5] - x[3],
                x[5] - x[4]]
        return cons[i]

    def callback(self, x):
        print("Did an iteration at ", x)


T_heat_reject = 305.
UAgoal = 100
xB = [400, 1, T_heat_reject, 3, T_heat_reject, 5, 285, 4, T_heat_reject, 0.15]
bdry = system_aqua1.makeBoundary(xB)
P = Problem_2_1(bdry, UAgoal)
xC = numpy.array([0.51284472, 277.97717012, 312.16427764, 313.6952877,
               310.24856734, 374.14020482])
opt = scipy.optimize.minimize(P.objective, xC, method="COBYLA",
                              constraints=P.constraints, callback=P.callback)
opt

### Trial 2.2

Just brainstorming ... As a brute-force backup, I would like to have an optimize routine that is a gradient-based method where the gradient calculation and line-search algorithms simply react to passing over the feasible boundary using exceptions instead of constraint functions. (This matters because the total UA constraint function can only be computed when the feasible heat exchange constraints are already satisfied, so in some cases it would have to return infinite values.) In other words, this would be an adaptive step size method. In pseudo-code,

In [ ]:
def optimize(fun,x0,max_iter = 100):
    x = x0.copy()
    for iter in range(max_iter):
        if step == 0:
            break
        G, H = calcGradient(fun, x)
        dx = calcLineSearch(fun, x, G, H)
        x = x + dx
        f = fun(x)
    return x, f

def calcGradient(fun, x0, max_iter = 100):
    max_iter = 100
    # Initial step sizes by component
    deltax = ones_like(x0)
    # Function values to the right
    fplus = zeros_like(x0)
    # Function values to the left
    fminus = zeros_like(x0)
    f0 = fun(x0)
    for j, _ in enumerate(deltax):
        for i in range(max_iter):
            dx = numpy.diag(deltax)[j]
            try:
                xplus = x0 + dx
                xminus = x0 - dx
                fplus[j] = fun(xplus)
                fminus[j] = fun(xminus)
                break
            except:
                deltax[j] *= 0.5
    grad, hess = calcGradHelper(f0, deltax, fplus, fminus) # TODO
    return grad, hess

def calcLineSearch(fun, x0, G, H):
    max_iter = 100
    step = 0
    direction = lshelper1(x0, G, H)
    for i in range(max_iter):
        try:
            lshelper(f
        except:
            step *= 0.5
    return step * direction

#### Aside
Furthermore, here is another related strategy. I could optimize first without the constraint on total UA. That would give me a feasible system. Then, given the desired total UA, I could have a feasible starting point by then turning down the mass flow rate using a linesearch on that parameter.

### Trial 2.3

Another idea (suggested in documention of early trials) is to do something different with the heat exchange feasibility constraints, utilizing energy imbalance. The final goal would be to allow UA values as inputs, and optimize the cooling capacity, as intended. An intermediate goal may be to simply demonstrate a solver with UA values as inputs (using some optimization routine to determine the temperature points that achieve the specified UA values).

# Scratch work

In [15]:
a=5
for i in range(9999):
    a*=0.5
    if a==0:
        break
i,a

(1076, 0.0)

In [25]:
gradient_step = numpy.array([1.,2.,3.,4.])
for j, _ in enumerate(gradient_step):
    for i in range(2):
        gradient_step[j] *= 0.5
gradient_step

array([ 0.25,  0.5 ,  0.75,  1.  ])

In [30]:
numpy.diag(gradient_step)[3]

array([ 0.,  0.,  0.,  1.])

In [50]:
%%html
<pre id="TOC-markdown">TOC will be here</pre>
<script>
$("#TOC-markdown").html(
    $('h1,h2,h3,h4').map(function(){return "  ".repeat($(this).prop("tagName")[1]) + "- ["+$(this).text()+"](" + $(this).children().attr("href") + ")";}).get().join("\n")
    );
</script>